## Interaktiv Phi 3 Mini 4K Instruktionschatbot med Whisper

### Introduktion:
Den interaktiva Phi 3 Mini 4K Instruktionschatbot är ett verktyg som låter användare interagera med Microsoft Phi 3 Mini 4K instruktionsdemo via text- eller ljudinmatning. Chatboten kan användas för en mängd olika uppgifter, såsom översättning, väderuppdateringar och allmän informationsinsamling.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Skapa din Huggingface Access Token

Skapa en ny token  
Ange ett nytt namn  
Välj skrivbehörigheter  
Kopiera token och spara den på en säker plats


Följande Python-kod utför två huvudsakliga uppgifter: importera modulen `os` och sätta en miljövariabel.

1. Importera modulen `os`:
   - Modulen `os` i Python ger ett sätt att interagera med operativsystemet. Den gör det möjligt att utföra olika operativsystemrelaterade uppgifter, såsom att komma åt miljövariabler, arbeta med filer och kataloger, etc.
   - I denna kod importeras modulen `os` med hjälp av `import`-satsen. Denna sats gör funktionaliteten i modulen `os` tillgänglig för användning i det aktuella Python-skriptet.

2. Sätta en miljövariabel:
   - En miljövariabel är ett värde som kan nås av program som körs på operativsystemet. Det är ett sätt att lagra konfigurationsinställningar eller annan information som kan användas av flera program.
   - I denna kod sätts en ny miljövariabel med hjälp av ordboken `os.environ`. Nyckeln i ordboken är `'HF_TOKEN'`, och värdet tilldelas från variabeln `HUGGINGFACE_TOKEN`.
   - Variabeln `HUGGINGFACE_TOKEN` definieras precis ovanför denna kodsnutt och tilldelas ett strängvärde `"hf_**************"` med hjälp av syntaxen `#@param`. Denna syntax används ofta i Jupyter-notebooks för att tillåta användarinmatning och parameterkonfiguration direkt i notebook-gränssnittet.
   - Genom att sätta miljövariabeln `'HF_TOKEN'` kan den nås av andra delar av programmet eller andra program som körs på samma operativsystem.

Sammanfattningsvis importerar denna kod modulen `os` och sätter en miljövariabel med namnet `'HF_TOKEN'` med det värde som tillhandahålls i variabeln `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Den här kodsnutten definierar en funktion som heter clear_output, som används för att rensa utdata från den aktuella cellen i Jupyter Notebook eller IPython. Låt oss gå igenom koden och förstå dess funktionalitet:

Funktionen clear_output tar en parameter som heter wait, vilket är en boolesk variabel. Som standard är wait satt till False. Denna parameter avgör om funktionen ska vänta tills ny utdata är tillgänglig för att ersätta den befintliga utdatan innan den rensas.

Själva funktionen används för att rensa utdata från den aktuella cellen. I Jupyter Notebook eller IPython, när en cell genererar utdata, såsom utskriven text eller grafiska diagram, visas den utdatan under cellen. Funktionen clear_output gör det möjligt att rensa bort den utdatan.

Implementeringen av funktionen tillhandahålls inte i kodsnutten, vilket indikeras av ellipsen (...). Ellipsen representerar en platshållare för den faktiska koden som utför rensningen av utdatan. Implementeringen av funktionen kan innebära interaktion med Jupyter Notebook- eller IPython-API:et för att ta bort den befintliga utdatan från cellen.

Sammanfattningsvis erbjuder denna funktion ett praktiskt sätt att rensa utdata från den aktuella cellen i Jupyter Notebook eller IPython, vilket gör det enklare att hantera och uppdatera den visade utdatan under interaktiva kodningssessioner.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Utför text-till-tal (TTS) med Edge TTS-tjänsten. Låt oss gå igenom relevanta funktionsimplementeringar en i taget:

1. `calculate_rate_string(input_value)`: Den här funktionen tar ett inmatningsvärde och beräknar hastighetssträngen för TTS-rösten. Inmatningsvärdet representerar den önskade talhastigheten, där ett värde på 1 motsvarar normal hastighet. Funktionen beräknar hastighetssträngen genom att subtrahera 1 från inmatningsvärdet, multiplicera det med 100 och sedan bestämma tecknet baserat på om inmatningsvärdet är större än eller lika med 1. Funktionen returnerar hastighetssträngen i formatet "{sign}{rate}".

2. `make_chunks(input_text, language)`: Den här funktionen tar en inmatningstext och ett språk som parametrar. Den delar upp inmatningstexten i delar baserat på språkets specifika regler. I denna implementation, om språket är "English", delar funktionen texten vid varje punkt (".") och tar bort eventuella inledande eller avslutande mellanslag. Den lägger sedan till en punkt till varje del och returnerar den filtrerade listan av delar.

3. `tts_file_name(text)`: Den här funktionen genererar ett filnamn för TTS-ljudfilen baserat på inmatningstexten. Den utför flera transformationer på texten: tar bort en avslutande punkt (om den finns), konverterar texten till små bokstäver, tar bort inledande och avslutande mellanslag och ersätter mellanslag med understreck. Den trunkerar sedan texten till maximalt 25 tecken (om den är längre) eller använder hela texten om den är tom. Slutligen genererar den en slumpmässig sträng med hjälp av [`uuid`] och kombinerar den med den trunkerade texten för att skapa filnamnet i formatet "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Den här funktionen slår samman flera ljudfiler till en enda ljudfil. Den tar en lista med ljudfilsvägar och en utmatningsväg som parametrar. Funktionen initierar ett tomt `AudioSegment`-objekt kallat [`merged_audio`]. Den itererar sedan genom varje ljudfilsväg, laddar ljudfilen med metoden `AudioSegment.from_file()` från biblioteket `pydub` och lägger till den aktuella ljudfilen till [`merged_audio`]-objektet. Slutligen exporterar den det sammanslagna ljudet till den angivna utmatningsvägen i MP3-format.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Den här funktionen utför TTS-operationen med Edge TTS-tjänsten. Den tar en lista med textdelar, talhastigheten, röstnamnet och sparvägen som parametrar. Om antalet delar är större än 1 skapar funktionen en katalog för att lagra de individuella ljudfilerna för varje del. Den itererar sedan genom varje del, konstruerar ett Edge TTS-kommando med hjälp av funktionen `calculate_rate_string()`, röstnamnet och texten för delen, och kör kommandot med funktionen `os.system()`. Om kommandot körs framgångsrikt lägger den till sökvägen för den genererade ljudfilen till en lista. Efter att ha bearbetat alla delar slår den samman de individuella ljudfilerna med funktionen `merge_audio_files()` och sparar det sammanslagna ljudet till den angivna sparvägen. Om det bara finns en del genererar den direkt Edge TTS-kommandot och sparar ljudet till sparvägen. Slutligen returnerar den sparvägen för den genererade ljudfilen.

6. `random_audio_name_generate()`: Den här funktionen genererar ett slumpmässigt ljudfilnamn med hjälp av [`uuid`]-modulen. Den genererar ett slumpmässigt UUID, konverterar det till en sträng, tar de första 8 tecknen, lägger till ".mp3"-tillägget och returnerar det slumpmässiga ljudfilnamnet.

7. `talk(input_text)`: Den här funktionen är huvudfunktionen för att utföra TTS-operationen. Den tar en inmatningstext som parameter. Den kontrollerar först längden på inmatningstexten för att avgöra om det är en lång mening (större än eller lika med 600 tecken). Baserat på längden och värdet på variabeln `translate_text_flag` avgör den språket och genererar listan med textdelar med funktionen `make_chunks()`. Den genererar sedan en sparväg för ljudfilen med funktionen `random_audio_name_generate()`. Slutligen anropar den funktionen `edge_free_tts()` för att utföra TTS-operationen och returnerar sparvägen för den genererade ljudfilen.

Sammanfattningsvis arbetar dessa funktioner tillsammans för att dela upp inmatningstexten i delar, generera ett filnamn för ljudfilen, utföra TTS-operationen med Edge TTS-tjänsten och slå samman de individuella ljudfilerna till en enda ljudfil.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementeringen av två funktioner: convert_to_text och run_text_prompt, samt deklarationen av två klasser: str och Audio.

Funktionen convert_to_text tar en audio_path som indata och transkriberar ljudet till text med hjälp av en modell som heter whisper_model. Funktionen kontrollerar först om gpu-flaggan är satt till True. Om den är det används whisper_model med vissa parametrar som word_timestamps=True, fp16=True, language='English' och task='translate'. Om gpu-flaggan är False används whisper_model med fp16=False. Den resulterande transkriptionen sparas sedan i en fil som heter 'scan.txt' och returneras som text.

Funktionen run_text_prompt tar ett meddelande och en chat_history som indata. Den använder funktionen phi_demo för att generera ett svar från en chatbot baserat på det givna meddelandet. Det genererade svaret skickas sedan till funktionen talk, som konverterar svaret till en ljudfil och returnerar filens sökväg. Klassen Audio används för att visa och spela upp ljudfilen. Ljudet visas med hjälp av display-funktionen från modulen IPython.display, och Audio-objektet skapas med parametern autoplay=True, så att ljudet börjar spela automatiskt. Chat_history uppdateras med det givna meddelandet och det genererade svaret, och en tom sträng samt den uppdaterade chat_history returneras.

Klassen str är en inbyggd klass i Python som representerar en sekvens av tecken. Den tillhandahåller olika metoder för att manipulera och arbeta med strängar, såsom capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill och fler. Dessa metoder gör det möjligt att utföra operationer som att söka, ersätta, formatera och manipulera strängar.

Klassen Audio är en anpassad klass som representerar ett ljudobjekt. Den används för att skapa en ljudspelare i Jupyter Notebook-miljön. Klassen accepterar olika parametrar såsom data, filename, url, embed, rate, autoplay och normalize. Parametern data kan vara en numpy-array, en lista med samples, en sträng som representerar ett filnamn eller URL, eller rå PCM-data. Parametern filename används för att specificera en lokal fil att ladda ljuddata från, och parametern url används för att specificera en URL att ladda ner ljuddata från. Parametern embed avgör om ljuddata ska bäddas in med en data-URI eller refereras från den ursprungliga källan. Parametern rate specificerar samplingsfrekvensen för ljuddata. Parametern autoplay avgör om ljudet ska börja spela automatiskt. Parametern normalize specificerar om ljuddata ska normaliseras (omskalas) till det maximala möjliga intervallet. Klassen Audio tillhandahåller också metoder som reload för att ladda om ljuddata från fil eller URL, och attribut som src_attr, autoplay_attr och element_id_attr för att hämta motsvarande attribut för ljudelementet i HTML.

Sammanfattningsvis används dessa funktioner och klasser för att transkribera ljud till text, generera ljudsvar från en chatbot och visa samt spela upp ljud i Jupyter Notebook-miljön.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
